# Data Preprocessing for Task 4: Utilization Prediction

In [1]:
import pandas as pd
import ast
import numpy as np
from datetime import datetime

## Weather Burbank Airport Dataset

Read in the preprocessed data.

In [2]:
Weather_set_preprocessed = pd.read_csv('weather_burbank_airport_preprocessed.csv', parse_dates=['timestamp'])
Weather_set_preprocessed['timestamp'] = pd.to_datetime(Weather_set_preprocessed['timestamp']).dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles')
Weather_set_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29244 entries, 0 to 29243
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype                              
---  ------                   --------------  -----                              
 0   timestamp                29244 non-null  datetime64[ns, America/Los_Angeles]
 1   temperature              29244 non-null  float64                            
 2   cloud_cover              29244 non-null  float64                            
 3   cloud_cover_description  29244 non-null  object                             
 4   pressure                 29244 non-null  float64                            
 5   windspeed                29244 non-null  float64                            
 6   precipitation            29244 non-null  float64                            
 7   felt_temperature         29244 non-null  float64                            
dtypes: datetime64[ns, America/Los_Angeles](1), float64(6), object(1)
me

Calculate hourly values.

In [3]:
# drop columns that are not needed
Weather_set_preprocessed = Weather_set_preprocessed.drop(['cloud_cover', 'cloud_cover_description','pressure', 'windspeed', 'felt_temperature'], axis=1)

# create hourly dataframe
Weather_set_preprocessed['date'] = Weather_set_preprocessed['timestamp'].dt.date
Weather_set_preprocessed['hour'] = Weather_set_preprocessed['timestamp'].dt.hour

start_timestamp = pd.to_datetime(Weather_set_preprocessed['timestamp'].min())
end_timestamp = pd.to_datetime(Weather_set_preprocessed['timestamp'].max())
days_unique = pd.date_range(start=start_timestamp, end=end_timestamp, freq='D').date

days = np.repeat(days_unique,24)
hours = list(range(0,24))
hours = hours*len(days_unique)

Weather_set_hourly = pd.DataFrame()
Weather_set_hourly['date'] = days
Weather_set_hourly['hour'] = hours

# add columns temperature and precipitation
temperature = []
precipitation = []
for index, row in Weather_set_hourly.iterrows():
    searched_day = row['date']
    searched_hour = row['hour']
    temperatures_of_hour = Weather_set_preprocessed[(Weather_set_preprocessed['date']==searched_day) & (Weather_set_preprocessed['hour']==searched_hour)]['temperature']
    temperature.append(temperatures_of_hour.mean())
    precipitation_of_hour = Weather_set_preprocessed[(Weather_set_preprocessed['date']==searched_day) & (Weather_set_preprocessed['hour']==searched_hour)]['precipitation']
    precipitation.append(precipitation_of_hour.mean())

Weather_set_hourly['temperature'] = temperature
Weather_set_hourly['precipitation'] = precipitation

# fill in missing values
def fill_in_mean_of_day(df,column):
    for index, row in df.iterrows():
        value = row[column]
        if pd.isna(value):
            day_of_null = row['date']
            mean_of_day = df[df['date']==day_of_null][column].mean()
            df.loc[index, column] = mean_of_day
    return df  

Weather_set_hourly = fill_in_mean_of_day(Weather_set_hourly, 'temperature')
Weather_set_hourly = fill_in_mean_of_day(Weather_set_hourly, 'precipitation')
Weather_set_hourly['month'] = pd.to_datetime(Weather_set_hourly['date']).dt.month
Weather_set_hourly['dayOfTheWeek'] = pd.to_datetime(Weather_set_hourly['date']).dt.dayofweek


Weather_set_hourly.info()
Weather_set_hourly

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           26304 non-null  object 
 1   hour           26304 non-null  int64  
 2   temperature    26280 non-null  float64
 3   precipitation  26280 non-null  float64
 4   month          26304 non-null  int32  
 5   dayOfTheWeek   26304 non-null  int32  
dtypes: float64(2), int32(2), int64(1), object(1)
memory usage: 1.0+ MB


,date,hour,temperature,precipitation,month,dayOfTheWeek
0,2018-01-01,0,9.0,0.0,1,0
1,2018-01-01,1,9.0,0.0,1,0
2,2018-01-01,2,9.0,0.0,1,0
3,2018-01-01,3,9.0,0.0,1,0
4,2018-01-01,4,8.0,0.0,1,0
...,...,...,...,...,...,...
26299,2020-12-31,19,13.0,0.0,12,3
26300,2020-12-31,20,12.0,0.0,12,3
26301,2020-12-31,21,12.0,0.0,12,3
26302,2020-12-31,22,11.0,0.0,12,3


## Charging Sessions Dataset

Read in the preprocessed data.

In [4]:
Charging_set_preprocessed = pd.read_csv('charging_sessions_preprocessed.csv', parse_dates=['connectionTime','disconnectTime','doneChargingTime'])
Charging_set_preprocessed['connectionTime'] = pd.to_datetime(Charging_set_preprocessed['connectionTime']).dt.tz_convert('America/Los_Angeles')
Charging_set_preprocessed['disconnectTime'] = pd.to_datetime(Charging_set_preprocessed['disconnectTime']).dt.tz_convert('America/Los_Angeles')
Charging_set_preprocessed['doneChargingTime'] = pd.to_datetime(Charging_set_preprocessed['doneChargingTime']).dt.tz_convert('America/Los_Angeles')
Charging_set_preprocessed['totalConnectionTime'] = pd.to_timedelta(Charging_set_preprocessed['totalConnectionTime'])
Charging_set_preprocessed['totalChargingTime'] = pd.to_timedelta(Charging_set_preprocessed['totalChargingTime'])
Charging_set_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60921 entries, 0 to 60920
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype                              
---  ------               --------------  -----                              
 0   id                   60921 non-null  object                             
 1   connectionTime       60921 non-null  datetime64[ns, America/Los_Angeles]
 2   disconnectTime       60921 non-null  datetime64[ns, America/Los_Angeles]
 3   doneChargingTime     60921 non-null  datetime64[ns, America/Los_Angeles]
 4   kWhDelivered         60921 non-null  float64                            
 5   sessionID            60921 non-null  object                             
 6   siteID               60921 non-null  int64                              
 7   spaceID              60921 non-null  object                             
 8   stationID            60921 non-null  object                             
 9   userID               44636 n

Calculate hourly values.

In [5]:
# create hourly dataframe
start_timestamp = pd.to_datetime(Weather_set_preprocessed['timestamp'].min())
end_timestamp = pd.to_datetime(Weather_set_preprocessed['timestamp'].max())
hourly_timestamps = pd.date_range(start=start_timestamp, end=end_timestamp, freq='H')
Charging_set_hourly = pd.DataFrame({
    'time_id': range(1, len(hourly_timestamps) + 1),
    'hourly_timestamp': hourly_timestamps
})
Charging_set_hourly['date'] = Charging_set_hourly['hourly_timestamp'].dt.date
Charging_set_hourly['month'] = Charging_set_hourly['hourly_timestamp'].dt.month
Charging_set_hourly['year'] = Charging_set_hourly['hourly_timestamp'].dt.year
Charging_set_hourly['hour'] = Charging_set_hourly['hourly_timestamp'].dt.hour
Charging_set_hourly['dayOfTheWeek'] = Charging_set_hourly['hourly_timestamp'].dt.dayofweek

# add utilization of site 1 and site 2
sessions_count_site1_utilization = []
sessions_count_site2_utilization = []

for timestamp in Charging_set_hourly['hourly_timestamp']:
    # filter sessions where the timestamp is within the connectionTime and disconnectTime range
    sessions_site1_utilization = Charging_set_preprocessed[
        (Charging_set_preprocessed['connectionTime'] < timestamp) &
        (Charging_set_preprocessed['doneChargingTime'] > timestamp) &
        (Charging_set_preprocessed['siteID'] == 1)
    ]
    sessions_site2_utilization = Charging_set_preprocessed[
        (Charging_set_preprocessed['connectionTime'] < timestamp) &
        (Charging_set_preprocessed['doneChargingTime'] > timestamp) &
        (Charging_set_preprocessed['siteID'] == 2)
    ]
    sessions_count_site1_utilization.append(len(sessions_site1_utilization))
    sessions_count_site2_utilization.append(len(sessions_site2_utilization))

Charging_set_hourly['site1_utilization'] = sessions_count_site1_utilization
Charging_set_hourly['site2_utilization'] = sessions_count_site2_utilization

total_number_stations_sites = Charging_set_preprocessed.groupby(['siteID'])['stationID'].nunique()

Charging_set_hourly['site1_utilization_percentage'] = Charging_set_hourly['site1_utilization'] / total_number_stations_sites.loc[1]
Charging_set_hourly['site2_utilization_percentage'] = Charging_set_hourly['site2_utilization'] / total_number_stations_sites.loc[2]

Charging_set_hourly

,time_id,hourly_timestamp,date,month,year,hour,dayOfTheWeek,site1_utilization,site2_utilization,site1_utilization_percentage,site2_utilization_percentage
0,1,2018-01-01 00:53:00-08:00,2018-01-01,1,2018,0,0,0,0,0.0,0.0
1,2,2018-01-01 01:53:00-08:00,2018-01-01,1,2018,1,0,0,0,0.0,0.0
2,3,2018-01-01 02:53:00-08:00,2018-01-01,1,2018,2,0,0,0,0.0,0.0
3,4,2018-01-01 03:53:00-08:00,2018-01-01,1,2018,3,0,0,0,0.0,0.0
4,5,2018-01-01 04:53:00-08:00,2018-01-01,1,2018,4,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
26299,26300,2020-12-31 19:53:00-08:00,2020-12-31,12,2020,19,3,0,0,0.0,0.0
26300,26301,2020-12-31 20:53:00-08:00,2020-12-31,12,2020,20,3,0,0,0.0,0.0
26301,26302,2020-12-31 21:53:00-08:00,2020-12-31,12,2020,21,3,0,0,0.0,0.0
26302,26303,2020-12-31 22:53:00-08:00,2020-12-31,12,2020,22,3,0,0,0.0,0.0


## Merged Dataset

In [9]:
Input_hourly = pd.merge(Charging_set_hourly, Weather_set_hourly, on=['date', 'hour', 'month', 'dayOfTheWeek'])
Input_hourly = Input_hourly.drop(['time_id', 'hourly_timestamp','year'], axis=1)
Input_hourly.info()
Input_hourly
Input_hourly.to_csv('input_hourly.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          26304 non-null  object 
 1   month                         26304 non-null  int32  
 2   hour                          26304 non-null  int32  
 3   dayOfTheWeek                  26304 non-null  int32  
 4   site1_utilization             26304 non-null  int64  
 5   site2_utilization             26304 non-null  int64  
 6   site1_utilization_percentage  26304 non-null  float64
 7   site2_utilization_percentage  26304 non-null  float64
 8   temperature                   26280 non-null  float64
 9   precipitation                 26280 non-null  float64
dtypes: float64(4), int32(3), int64(2), object(1)
memory usage: 1.7+ MB
